In [42]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import glob

In [43]:
#Cargando archivo del mapping
sntc_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\4-SNTC_Princing_List.xlsx",sheet_name="Output").fillna(0)
sntc_mapping = sntc_mapping[["Product SKU ","SNT"]]

# Functions

In [44]:
%%time
#Definiendo funciones
#Definiendo funcion para calculo de LDos flag
def LDoS_flag(data):
    list=[]
    for x in data:
        if x >= dt.today():
            list.append('N')
        else:
            list.append('Y')
    return list

# Funcion calcula el valor del IB value
def IB_Value(path, sntc_mapping):
    '''
    La funcion recibe la ruta de un archivo excel y calcula el IB value
    entradas:
        path: Ruta del archivo.
    salidas:
        
    '''
    
    #Leyendo archivo de mapping - CAMBIAR LA RUTA A DONDE ESTE EL ARCHIVO -

    
    #Leyendo archivo de IB
    ib = pd.read_csv(path)
    
    
    #Haciendo merge a los dos archivos para traer las columnas del valor del SNT a la tabla ib
   
    merge = ib.merge(sntc_mapping,right_on="Product SKU ",left_on="Product ID",how="left")
    
    #Transformando las fechas para calcular la columna LDos flag
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('/','-'))    
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('nan', '2100-01-01 00:00:00'))
    fechas = [dt.strptime(str(i)[:-5],'%m-%d-%Y') if len(i)<16 else dt.fromisoformat(str(i)) for i in merge['LDoS']]
    merge['LDoS'] = fechas
    
    #Calculando columna LDos flag
    merge['LDoS Flag']=LDoS_flag(merge['LDoS'])
    merge[['LDoS','LDoS Flag']][merge['LDoS Flag']=='N']

    #Ejecutando los filtros a la tabla
    merge=merge[merge['Coverage']=='NOT COVERED']

    #merge=merge[(merge["Asset Type"]=="Hardware")]

    merge=merge[(merge['Product Type']!= 'APPLIANCE') & (merge['Product Type']!='CABLE') & (merge['Product Type']!='APPSWIND') & (merge['Product Type']!='POWER') & (merge['Product Type']!='DOC')]

    merge=merge[(merge['LDoS Flag']=='N')]

    #merge = merge[(merge['Business Entity Name'] != 'Other')]

    # Calculando el ib value

    merge["IB Oppty"] = merge["Item Quantity"]*merge["SNT"]/1000
    final = merge.iloc[:, np.r_[0,-1]].groupby(merge.columns[0]).sum()
    
    if len(final.index) > 1:
        lista_savid.append(final.index[0])
        lista_ib.append(round(final['IB Oppty'].sum(),1))
        return print('package #',final.index[0],'done, IB Oppty = ', round(final['IB Oppty'].sum(),1))
    else:
        lista_savid.append(final.index[0])
        lista_ib.append(round(final['IB Oppty'].to_list()[0],1))
        return print('package # ',final.index[0],' done, IB Oppty = ', round(final['IB Oppty'].to_list()[0],1))

    

Wall time: 0 ns


# Testing

In [45]:
# Antes de iniciar se deben cargar las url de los archivos en una lista
## Cargando las URL de los archivos desde la carpeta - CAMBIAR LA RUTA A LA CARPETA DONDE ESTAN LOS ARCHIVOS

path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\*'
img_number = 1

#El codigo extrae la url de cada imagen en el pc
listaarchivos = []
for file in glob.glob(path):
    print(file+'\Data\IB\IB.csv')
    listaarchivos.append(file+'\Data\IB\IB.csv')

C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203684898_COMMUNITYHEALT_2022_2_16\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203694540_HENRYFORDHEAL_2022_2_16\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203703214_DTEENERGYCO_2022_2_16\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203706832_CARESOURCE_2022_2_16\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203711476_KEYCORP_2022_2_16\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203711478_MARATHONPETROL_2022_2_16\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-16\OP_Americas_SAV ID_203712334_BIGLOTS_2022

In [46]:
%%time
# Ahora se calculan todos lo IB value de los archivos mediante un ciclo
# Calculando todos los IB_Value en bache
#Las listas se generan afuera y en ellas se almacenan los datos calculados
lista_ib=[]
lista_savid=[]
for i in listaarchivos:
    IB_Value(i, sntc_mapping)

package #  203684898  done, IB Oppty =  386.1
package #  203694540  done, IB Oppty =  210.1
package #  203703214  done, IB Oppty =  725.3
package #  203706832  done, IB Oppty =  134.1
package #  203711476  done, IB Oppty =  299.8
package #  203711478  done, IB Oppty =  3848.8
package #  203712334  done, IB Oppty =  366.4
package #  203738629  done, IB Oppty =  201.2
package #  203751115  done, IB Oppty =  1075.2
package #  203774795  done, IB Oppty =  2143.4
package #  203779104  done, IB Oppty =  8480.1
package #  203794935  done, IB Oppty =  736.2
package #  203838669  done, IB Oppty =  122.0
package #  203881924  done, IB Oppty =  533.2
Wall time: 772 ms


In [6]:
#pasando los datos calculados a excel
IB_calculated = pd.DataFrame(zip(lista_savid,lista_ib), columns=['ID', 'IB Oppty'])
IB_calculated.to_excel(r'values dilia.xlsx')
IB_calculated

,ID,IB Oppty
0,22028,811.5
1,2646353,818.8
2,266313198,126.8
3,37455866,1095.9
4,37455866,785.7
5,43319089,183.5


# Excel files merge

In [62]:
path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\*'
img_number = 1

#El codigo extrae la url de cada imagen en el pc
listaarchivosexcel = []
for file in glob.glob(path):
    print(file)
    listaarchivosexcel.append(file)

C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\dec_01.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\dec_02.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\dec_03.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\nov_22.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\nov_23.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\nov_24.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\nov_25.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\nov_29.xlsx
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Proyectos Python\SNT Value\check exel\nov_30.xlsx


In [65]:
df1 =pd.DataFrame()
for i in listaarchivosexcel:
    df =pd.read_excel(i)
    df1 = pd.concat([df,df1], axis=0)
df1.to_excel(r'df1.xlsx')
df1

,Unnamed: 0,ID,IB Oppty
0,0,203686953,73.9
1,1,203694558,459.1
2,2,203695027,1331.3
3,3,203704594,185.2
4,4,203715617,877.9
...,...,...,...
15,15,4242933,73.6
16,16,6374115,790.9
17,17,6701570,163.6
18,18,9256005,899.3


# One Account

In [10]:

path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\December\2021-12-13\OP_APJC_GU ID_147183_NIPPONTELEGRAP_2021_12_13\Data\IB\IB.csv'
ib = pd.read_csv(path)
lista_savid = []
lista_ib = []  

#Haciendo merge a los dos archivos para traer las columnas del valor del SNT a la tabla ib

merge = ib.merge(sntc_mapping,right_on="Product SKU ",left_on="Product ID",how="left")


#Transformando las fechas para calcular la columna LDos flag
merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('/','-'))    
merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('nan', '2100-01-01 00:00:00'))

fechas = []
for i in merge['LDoS']:
    if len(i) < 16:
        i = i[:-5]   
        i = dt.strptime(i, '%m-%d-%Y')
        fechas.append(i)     
    else:
        i = dt.fromisoformat(i)
        fechas.append(i)


merge['LDoS'] = fechas

#Calculando columna LDos flag
merge['LDoS Flag']=LDoS_flag(merge['LDoS'])
merge[['LDoS','LDoS Flag']][merge['LDoS Flag']=='N']

#Ejecutando los filtros a la tabla
merge=merge[merge['Coverage']=='NOT COVERED']

#merge=merge[(merge["Asset Type"]=="Hardware")]

merge=merge[(merge['Product Type']!= 'APPLIANCE') & (merge['Product Type']!='CABLE') & (merge['Product Type']!='APPSWIND') & (merge['Product Type']!='POWER') & (merge['Product Type']!='DOC')]

merge=merge[(merge['LDoS Flag']=='N')]

#merge = merge[(merge['Business Entity Name'] != 'Other')]

# Calculando el ib value
lista_savid=[]
lista_ib=[]
merge["IB Oppty"] = merge["Item Quantity"]*merge["SNT"]/1000
final = merge.iloc[:, np.r_[0,-1]].groupby(merge.columns[0]).sum()

if len(final.index) > 1:
    lista_savid.append(final.index[0])
    lista_ib.append(round(final['IB Oppty'].sum(),1))
    print('package #',final.index[0],'done, IB Oppty = ', round(final['IB Oppty'].sum(),1))
else:
    lista_savid.append(final.index[0])
    lista_ib.append(round(final['IB Oppty'].to_list()[0],1))
    print('package # ',final.index[0],' done, IB Oppty = ', round(final['IB Oppty'].to_list()[0],1))



package # 203733555 done, IB Oppty =  18068.5


# More than One Account in IB file

In [4]:
path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Initiatives\FY22Q2 - High Potential Acoounts\HPA DATA2\*'

lista_sav_IB= []

for file in glob.glob(path):
    lista_sav_IB.append(file)
    
SAV_IB =pd.DataFrame()

for i in lista_sav_IB:
    df =pd.read_csv(i,dtype={'SAV ID':str,
                            'Instance Shipped Fiscal Year':str,
                            'Item Quantity':float,
                            'LDoS FY':str,
                            'Best Site ID':int,
                            'Contract Type':str,
                            'Covered Line Status':str,
                            'Contract Number':str,
                            'Service Brand Code':str,
                            "Asset List Amount":float},decimal=",")
    df.insert(loc=2, column='Best Site ID', value=-9999)
    df.insert(loc=3, column='Best Site Customer Name', value='UNKNOWN')
    SAV_IB = pd.concat([df,SAV_IB], axis=0)
IB_full = SAV_IB
IB_full

,SAV ID,SAV Name,Best Site ID,Best Site Customer Name,Best Site Sales Level 2 Name,Coverage,Contract Number,Covered Line Status,Contract Type,Contract Line End Quarter,...,SWSS Flag,Default Service List Price USD,Item Quantity,Annual Extended Contract Line List USD Amount,Annual Contract Line Net USD Amount,Annualized Extended Contract Line List USD Amount,Annualized Contract Line Net USD Amount,Contract Line List Price USD,Contract Line Net Price USD,Asset List Amount
0,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,512.01
1,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,2589.44
2,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,229.53
3,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,13064.70
4,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,423.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415803,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,COVERED,204218335,ACTIVE,PSUP,20251,...,N,165.0,1.0,479.85,86.276667,159.95,86.276667,479.85,258.83,2818.92
415804,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,3636.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,89863.95
415805,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,42889.88
415806,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,2882.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,60536.00


In [5]:
IB_copy = IB_full.copy()
lista_id = IB_copy.iloc[:,0].unique().tolist()
lista_ib=[]
lista_savid=[]

In [6]:
%%time   
#Leyendo archivo de IB
for j in lista_id:
    ib=IB_copy[IB_copy.iloc[:,0] == j]

    #Haciendo merge a los dos archivos para traer las columnas del valor del SNT a la tabla ib

    merge = ib.merge(sntc_mapping,right_on="Product SKU ",left_on="Product ID",how="left")

    #Transformando las fechas para calcular la columna LDos flag
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('/','-'))    
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('nan', '2100-01-01 00:00:00'))
    fechas = [dt.strptime(str(i)[:-5],'%m-%d-%Y') if len(i)<16 else dt.fromisoformat(str(i)) for i in merge['LDoS']]
    merge['LDoS'] = fechas

    #Calculando columna LDos flag
    merge['LDoS Flag']=LDoS_flag(merge['LDoS'])
    merge[['LDoS','LDoS Flag']][merge['LDoS Flag']=='N']

    #Ejecutando los filtros a la tabla
    merge=merge[merge['Coverage']=='NOT COVERED']

    #merge=merge[(merge["Asset Type"]=="Hardware")]

    merge=merge[(merge['Product Type']!= 'APPLIANCE') & (merge['Product Type']!='CABLE') & (merge['Product Type']!='APPSWIND') & (merge['Product Type']!='POWER') & (merge['Product Type']!='DOC')]

    merge=merge[(merge['LDoS Flag']=='N')]

    #merge = merge[(merge['Business Entity Name'] != 'Other')]

    # Calculando el ib value

    merge["IB Oppty"] = merge["Item Quantity"]*merge["SNT"]/1000
    final = merge.iloc[:, np.r_[0,-1]].groupby(merge.columns[0]).sum()

    if len(final.index) > 1:
        lista_savid.append(final.index[0])
        lista_ib.append(round(final['IB Oppty'].sum(),1))
        print('package #',final.index[0],'done, IB Oppty = ', round(final['IB Oppty'].sum(),1))
    else:
        lista_savid.append(final.index[0])
        lista_ib.append(round(final['IB Oppty'].to_list()[0],1))
        print('package # ',final.index[0],' done, IB Oppty = ', round(final['IB Oppty'].to_list()[0],1))

    

package #  222173994  done, IB Oppty =  112.7
package #  230366718  done, IB Oppty =  26.0
package #  203840606  done, IB Oppty =  201.3
package #  207593001  done, IB Oppty =  55.5
package #  283468065  done, IB Oppty =  55.2
package #  203772167  done, IB Oppty =  152.0
package #  203772245  done, IB Oppty =  304.7
package #  203711644  done, IB Oppty =  383.1
package #  203849413  done, IB Oppty =  384.6
package #  203701565  done, IB Oppty =  88.9
package #  283536694  done, IB Oppty =  106.0
package #  203743221  done, IB Oppty =  452.2
package #  233445185  done, IB Oppty =  681.9
package #  203785620  done, IB Oppty =  93.8
package #  203729711  done, IB Oppty =  208.5
package #  203853367  done, IB Oppty =  545.5
package #  203851345  done, IB Oppty =  92.3
package #  203730139  done, IB Oppty =  10.6
package #  215268553  done, IB Oppty =  16.7
package #  203793594  done, IB Oppty =  47.1
package #  203779032  done, IB Oppty =  111.6
package #  203701558  done, IB Oppty =  590

In [7]:
len(lista_savid)
len(lista_ib)
dft= pd.DataFrame()
dft['ID'] = lista_savid
dft['SNTC_Oppty'] = lista_ib
dft

,ID,SNTC_Oppty
0,222173994,112.7
1,230366718,26.0
2,203840606,201.3
3,207593001,55.5
4,283468065,55.2
...,...,...
1326,283714968,4140.2
1327,203797997,8385.4
1328,283715236,19801.9
1329,283714970,18514.4


In [8]:
dft.to_csv('SNTC Oppty - HPA-5-2-2022.csv')